In [3]:
import nest_asyncio
nest_asyncio.apply()

In [2]:
import collections

import numpy as np
import tensorflow as tf
import tensorflow_federated as tff

np.random.seed(0)

In [4]:
emnist_train, emnist_test = tff.simulation.datasets.emnist.load_data()

In [5]:
from tensorflow_federated.proto.v0 import computation_pb2 as pb


NUM_CLIENTS = 10
NUM_EPOCHS = 5
SHUFFLE_BUFFER = 100

def preprocess(dataset):

  def map_fn(element):
    return collections.OrderedDict(
        x=tf.reshape(element['pixels'], [-1, 784]),
        y=tf.reshape(element['label'], [-1, 1]))

  return dataset.repeat(NUM_EPOCHS).shuffle(SHUFFLE_BUFFER, seed=1).map(map_fn)

In [6]:
example_dataset = emnist_train.create_tf_dataset_for_client(
    emnist_train.client_ids[0])
preprocessed_example_dataset = preprocess(example_dataset)

In [7]:
class TestDataBackend(tff.framework.DataBackend):

  async def materialize(self, data, type_spec):
    client_id = int(data.uri[-1])
    client_dataset = emnist_train.create_tf_dataset_for_client(
        emnist_train.client_ids[client_id])
    return preprocess(client_dataset)

In [8]:
def ex_fn(
    device: tf.config.LogicalDevice) -> tff.framework.DataExecutor:
  return tff.framework.DataExecutor(
      tff.framework.EagerTFExecutor(device),
      data_backend=TestDataBackend())
factory = tff.framework.local_executor_factory(leaf_executor_fn=ex_fn)
ctx = tff.framework.ExecutionContext(executor_fn=factory)
tff.framework.set_default_context(ctx)

In [9]:
def create_keras_model():
  return tf.keras.models.Sequential([
      tf.keras.layers.InputLayer(input_shape=(784,)),
      tf.keras.layers.Dense(10, kernel_initializer='zeros'),
      tf.keras.layers.Softmax(),
  ])
  
def model_fn():
  keras_model = create_keras_model()
  return tff.learning.from_keras_model(
      keras_model,
      input_spec=preprocessed_example_dataset.element_spec,
      loss=tf.keras.losses.SparseCategoricalCrossentropy(),
      metrics=[tf.keras.metrics.SparseCategoricalAccuracy()])

In [10]:
iterative_process = tff.learning.build_federated_averaging_process(
    model_fn,
    client_optimizer_fn=lambda: tf.keras.optimizers.SGD(learning_rate=0.02),
    server_optimizer_fn=lambda: tf.keras.optimizers.SGD(learning_rate=1.0))

state = iterative_process.initialize()

In [11]:
element_type = tff.types.StructWithPythonType(
    preprocessed_example_dataset.element_spec,
    container_type=collections.OrderedDict)
dataset_type = tff.types.SequenceType(element_type)
dataset_type_proto = tff.framework.serialize_type(dataset_type)
# Sampling the first five clients in the EMNIST dataset.
arguments = [
  pb.Computation(data=pb.Data(uri=f'uri://{i}'), type=dataset_type_proto)
  for i in range(5)
]
data_handle = tff.framework.DataDescriptor(None, arguments, 
  tff.FederatedType(dataset_type, tff.CLIENTS), len(arguments))

In [12]:
state, metrics = iterative_process.next(state, data_handle)
print('round 1, metrics={}'.format(metrics))

round 1, metrics=OrderedDict([('broadcast', ()), ('aggregation', OrderedDict([('mean_value', ()), ('mean_weight', ())])), ('train', OrderedDict([('sparse_categorical_accuracy', 0.11625), ('loss', 12.6826515), ('num_examples', 2400), ('num_batches', 2400)]))])


In [13]:
NUM_ROUNDS = 11
for round_num in range(2, NUM_ROUNDS):
  state, metrics = iterative_process.next(state, data_handle)
  print('round {:2d}, metrics={}'.format(round_num, metrics))

round  2, metrics=OrderedDict([('broadcast', ()), ('aggregation', OrderedDict([('mean_value', ()), ('mean_weight', ())])), ('train', OrderedDict([('sparse_categorical_accuracy', 0.12375), ('loss', 10.283682), ('num_examples', 2400), ('num_batches', 2400)]))])
round  3, metrics=OrderedDict([('broadcast', ()), ('aggregation', OrderedDict([('mean_value', ()), ('mean_weight', ())])), ('train', OrderedDict([('sparse_categorical_accuracy', 0.17125), ('loss', 7.746421), ('num_examples', 2400), ('num_batches', 2400)]))])
round  4, metrics=OrderedDict([('broadcast', ()), ('aggregation', OrderedDict([('mean_value', ()), ('mean_weight', ())])), ('train', OrderedDict([('sparse_categorical_accuracy', 0.27625), ('loss', 5.787291), ('num_examples', 2400), ('num_batches', 2400)]))])
round  5, metrics=OrderedDict([('broadcast', ()), ('aggregation', OrderedDict([('mean_value', ()), ('mean_weight', ())])), ('train', OrderedDict([('sparse_categorical_accuracy', 0.37541667), ('loss', 4.511521), ('num_examp

In [14]:
preprocessed_example_dataset.element_spec

OrderedDict([('x', TensorSpec(shape=(1, 784), dtype=tf.float32, name=None)),
             ('y', TensorSpec(shape=(1, 1), dtype=tf.int32, name=None))])

In [15]:
model = create_keras_model()
print(model)
input_spec = [tf.TensorSpec(shape=(1,784), dtype=tf.float32, name=None), tf.TensorSpec(shape=(1,1), dtype=tf.int32, name=None)]
functional_model = tff.learning.models.functional_model_from_keras(keras_model=model, loss_fn=tf.keras.losses.SparseCategoricalCrossentropy(),input_spec=input_spec)
print(functional_model.input_spec[0])
tff.learning.models.save_functional_model(functional_model=functional_model, path='tmp.txt')


TensorSpec(shape=(1, 784), dtype=tf.float32, name=None)


INFO:tensorflow:Assets written to: tmp.txt/assets


INFO:tensorflow:Assets written to: tmp.txt/assets


In [16]:
import shutil
shutil.make_archive('tmp.zip', 'zip', 'tmp.txt')

'/home/teo/PySyft/notebooks/PySyTFF/tmp.zip.zip'

In [19]:
import zipfile
with zipfile.ZipFile('tmp.zip.zip', 'r') as zip_ref:
    zip_ref.extractall('tmp')

In [20]:
functional_model_reloaded = tff.learning.models.load_functional_model('tmp.txt')
functional_model_reloaded

In [21]:
new_model = tff.learning.models.model_from_functional(functional_model_reloaded)

In [22]:
new_model

In [28]:
iterative_process = tff.learning.build_unweighted_fed_avg(
    lambda: new_model,
    client_optimizer_fn=lambda: tf.keras.optimizers.SGD(learning_rate=0.02),
    server_optimizer_fn=lambda: tf.keras.optimizers.SGD(learning_rate=1.0))

state = iterative_process.initialize()

AttributeError: in user code:

    File "/home/teo/anaconda3/envs/PySyTFF/lib/python3.9/site-packages/tensorflow_federated/python/learning/optimizers/keras_optimizer.py", line 66, in mock_apply_gradients  *
        opt.apply_gradients([
    File "/home/teo/anaconda3/envs/PySyTFF/lib/python3.9/site-packages/keras/optimizer_v2/optimizer_v2.py", line 671, in apply_gradients  **
        return tf.__internal__.distribute.interim.maybe_merge_call(
    File "/home/teo/anaconda3/envs/PySyTFF/lib/python3.9/site-packages/keras/optimizer_v2/optimizer_v2.py", line 715, in _distributed_apply
        var.op.name):

    AttributeError: Tensor.op is meaningless when eager execution is enabled.
